In [3]:
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random

from sklearn.metrics import  classification_report
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve, auc

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

from sklearn import tree
from sklearn.tree import DecisionTreeClassifier

%matplotlib inline 

In [2]:
import importlib
import ids_common
importlib.reload(ids_common)

<module 'ids_common' from 'd:\\stuff\\univ\\ids\\ids_common.py'>

In [10]:
sample_koefs = {
# ATTACK
    'OS_Fingerprinting_attack.csv':1,
    'MITM_attack.csv':1,
    'Ransomware_attack.csv':1,
    'XSS_attack.csv':1,
    'Port_Scanning_attack.csv':1,
    'Backdoor_attack.csv':1,
    'Uploading_attack.csv':1,
    'SQL_injection_attack.csv':1,
    'Vulnerability_scanner_attack.csv':0.5,
    'DDoS_HTTP_Flood_attack.csv':0.5,
    'Password_attack.csv':0.3,
    'DDoS_TCP_SYN_Flood_attack.csv':0.1,
    'DDoS_ICMP_Flood_attack.csv':0.1,
    'DDoS_UDP_Flood_attack.csv':0.1,
# Normal
    'Modbus.csv':0.5,
    'Heart_Rate.csv':0.5,
    'phValue.csv':0.3,
    'Flame_Sensor.csv':0.25,
    'Distance.csv':0.25,
    'Soil_Moisture.csv':0.25,
    'IR_Receiver.csv':0.2,
    'Sound_Sensor.csv':0.2,
    'Temperature_and_Humidity.csv':0.2,
    'Water_Level.csv':0.2
}

In [43]:
sample_dir = './sampled_data/sample_1/'
if not os.path.exists(sample_dir):
    os.mkdir(sample_dir)

with open(sample_dir + 'koefs.txt', 'w') as f:
    for name, koef in sample_koefs.items():
        f.write(f"'{name}' : {koef},\n")


In [39]:
DATASET = 'dataset/Edge-IIoTset-dataset'
datasets_locations = []
for path, dirs, files in os.walk(DATASET):
    for file in files:
        if file.endswith('csv'):
            loc = f'{path}/{file}'
            if 'Selected dataset for ML and DL' not in loc:
                datasets_locations.append(loc)

In [42]:
#sample

total_size = 0

for ds_loc in datasets_locations:
    _, f = os.path.split(ds_loc)

    df = pd.read_csv(ds_loc, low_memory=False)
    koef = sample_koefs[f]
    df_sample = df.sample(frac=koef)

    print(f'{f} {sample_koefs[f]}: {df.shape} -> {df_sample.shape}')

    total_size += df_sample.shape[0]

    df_sample.to_csv(sample_dir + f'sampled_{koef}_' + f, index=False)

print(f'Total sample size: {total_size}')

Backdoor_attack.csv 1: (24862, 63) -> (24862, 63)
DDoS_HTTP_Flood_attack.csv 0.5: (229022, 63) -> (114511, 63)
DDoS_ICMP_Flood_attack.csv 0.1: (2914354, 63) -> (291435, 63)
DDoS_TCP_SYN_Flood_attack.csv 0.1: (2020120, 63) -> (202012, 63)
DDoS_UDP_Flood_attack.csv 0.1: (3201626, 63) -> (320163, 63)
MITM_attack.csv 1: (1229, 63) -> (1229, 63)
OS_Fingerprinting_attack.csv 1: (1001, 63) -> (1001, 63)
Password_attack.csv 0.3: (1053385, 63) -> (316016, 63)
Port_Scanning_attack.csv 1: (22564, 63) -> (22564, 63)
Ransomware_attack.csv 1: (10925, 63) -> (10925, 63)
SQL_injection_attack.csv 1: (51203, 63) -> (51203, 63)
Uploading_attack.csv 1: (37634, 63) -> (37634, 63)
Vulnerability_scanner_attack.csv 0.5: (145869, 63) -> (72934, 63)
XSS_attack.csv 1: (15915, 63) -> (15915, 63)
Distance.csv 0.25: (1143540, 63) -> (285885, 63)
Flame_Sensor.csv 0.25: (1070196, 63) -> (267549, 63)
Heart_Rate.csv 0.5: (165319, 63) -> (82660, 63)
IR_Receiver.csv 0.2: (1307778, 63) -> (261556, 63)
Modbus.csv 0.5: (159

In [44]:
# combine dataframes

df_combined = pd.DataFrame()
for ds_file in os.listdir(sample_dir):
    if 'sampled' in ds_file:
        df = pd.read_csv(sample_dir + ds_file, low_memory=False)
        df_combined = pd.concat([df_combined, df], ignore_index=True)

        print(f'Added: {ds_file} {df.shape}')

print(f'Combined df size: {df_combined.shape}')
df_combined.to_csv(sample_dir + f'combined_dataset.csv', index=False)


Added: sampled_0.1_DDoS_ICMP_Flood_attack.csv (291435, 63)
Added: sampled_0.1_DDoS_TCP_SYN_Flood_attack.csv (202012, 63)
Added: sampled_0.1_DDoS_UDP_Flood_attack.csv (320163, 63)
Added: sampled_0.25_Distance.csv (285885, 63)
Added: sampled_0.25_Flame_Sensor.csv (267549, 63)
Added: sampled_0.25_Soil_Moisture.csv (298194, 63)
Added: sampled_0.2_IR_Receiver.csv (261556, 63)
Added: sampled_0.2_Sound_Sensor.csv (302577, 63)
Added: sampled_0.2_Temperature_and_Humidity.csv (323144, 63)
Added: sampled_0.2_Water_Level.csv (459058, 63)
Added: sampled_0.3_Password_attack.csv (316016, 63)
Added: sampled_0.3_phValue.csv (224072, 63)
Added: sampled_0.5_DDoS_HTTP_Flood_attack.csv (114511, 63)
Added: sampled_0.5_Heart_Rate.csv (82660, 63)
Added: sampled_0.5_Modbus.csv (79751, 63)
Added: sampled_0.5_Vulnerability_scanner_attack.csv (72934, 63)
Added: sampled_1_Backdoor_attack.csv (24862, 63)
Added: sampled_1_MITM_attack.csv (1229, 63)
Added: sampled_1_OS_Fingerprinting_attack.csv (1001, 63)
Added: samp

In [45]:
df_combined['Attack_type'].value_counts()

Attack_type
Normal                   2584446
DDoS_UDP                  320163
Password                  316016
DDoS_ICMP                 291435
DDoS_TCP                  202012
DDoS_HTTP                 114511
Vulnerability_scanner      72934
SQL_injection              51203
Uploading                  37634
Backdoor                   24862
Port_Scanning              22564
XSS                        15915
Ransomware                 10925
MITM                        1229
OS_Fingerprinting           1001
Name: count, dtype: int64